# Augmentation Library Imgaug 설치

In [ ]:
!pip install imgaug

# 라이브러리 불러오기

In [1]:
import glob
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import xml.etree.ElementTree as ET
from PIL import Image
import os
%matplotlib inline

# 변수 세팅

In [2]:
label_info_path = 'classes.txt' # 텍스트 파일 만들고 label 을 순서대로 적어 놓자 
# aug folder path
aug_folder = 'data'
# img_file_path
img_file_path = aug_folder + "/*.jpg"
#xml_file_path 
xml_file_path = aug_folder + "/*.xml"
#txt_file_path
txt_file_path = aug_folder + "/*.txt"

#save_dir
save_dir = "aug"
#save_label
save_label = save_dir +"\\"+ "label"
#save_img
save_img = save_dir +"\\"+"img"

# Augmentation을 할 옵션들을 선택
augs = ["Add","Dropout","MotionBlur"]

#input format - xml, txt
input_format = 'txt'
#save format, yolo, xml 지금은 yolo만 구현
save_format = 'yolo'

#반복 횟수
iter_num = 2

## label mapping 중

In [3]:
labelmap ={}
labellist = []
with open(label_info_path, mode='r', encoding='utf-8') as f:
    label_list = f.readlines()
for i,label in enumerate(label_list):
    labelmap[label.replace('\n',"")] = i
    labellist.append(label.replace('\n',""))

# augmentation 할 파일 경로 리스트 세팅

In [6]:
## augmentation 할 파일 경로 리스트 세팅

img_path_list = glob.glob(img_file_path)
if input_format == 'xml':
    xml_path_list = glob.glob(xml_file_path)
elif input_format == 'txt':        
    txt_path_list = glob.glob(txt_file_path)

In [7]:
# img_path_list

In [8]:
# xml_path_list

In [9]:
# txt_path_list

# Augmentation 기능 매핑 및 추가

링크 타고 가서 어떤 효과인지는 찾아보세요 
https://imgaug.readthedocs.io/en/latest/source/overview/geometric.html


In [10]:
# aug 종류
#너무 많아서 손아파서 못쓰겠다 알아서 써라 


# augmentation 리스트 초기화
aug_list = []

if "Add" in augs:
    aug_list.append(iaa.Add((-40,40)))
if "AddElementwise" in augs:
    aug_list.append(iaa.AddElementwise((-40, 40)))
if "AddtiveGaussianNoise" in augs:
    aug_list.append(iaa.AddtiveGaussianNoise(scale=(0, 0.2*255), per_channel=True))
    aug_list.append(iaa.AddtiveGaussianNoise(scale=(0, 0.2*255)))
if "AddtiveLaplaceNoise" in augs:
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255)))
    aug_list.append(iaa.AddtiveLaplaceNoise(scale=(0,0.2*255), per_channel=True))
if "AddtivePoissonNoise" in augs:
    aug_list.append(iaa.AdditivePoissonNoise(scale=(0, 40)))
if "Multiply" in augs:
    aug_list.append(iaa.Multiply((0.5, 1.5)))
if "MultiplyElementwise" in augs:
    aug_list.append(iaa.Cutout(nb_iterations=(1,5), fill_mode="gaussian", fill_per_channel=True))
    aug_list.append(iaa.Cutout(fill_mode="gaussian", fill_per_channel=False))
if "Dropout" in augs:
    aug_list.append(iaa.Dropout(p=(0,0.2)))
    aug_list.append(iaa.Dropout(p=(0,0.2), per_channel=0.5))
if "CoarseDropout" in augs:
    aug_list.append(iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02,0.25)))
    aug_list.append(iaa.CoarseDropout(0.02, size_percent=0.15, per_channel=0.5))
if "ImpulseNoise" in augs:
    aug_list.append(iaa.ImpulseNoise(0.1))
if "SaltAndPepper" in augs:
    aug_list.append(iaa.SaltAndPepper(0.1), per_channel=True)
if "CoarseSaltAndPepper" in augs:
    aug_list.append(iaa.CoarseSaltAndPepper(
    0.05, size_percent=(0.01, 0.1), per_channel=True))
if "Invert" in augs:
    aug_list.append(iaa.Invert(0.25, per_channel=0.5))
if "Solarize" in augs:
    aug_list.append(iaa.Solarize(0.5, threshold=(32,128)))
if "JpegComppression" in augs:
    aug_list.append(iaa.JpegCompression(compression=(70,99)))
if "Cartoon" in augs:
    aug_list.append(iaa.Cartoon())
if "GaussianBlur" in augs:
    aug_list.append(iaa.GaussianBlur(sigma=(0.0,3.0)))
if "AverageBlur" in augs:
    aug_list.append(iaa.AverageBlur(k=(2,11)))
if "MedianBlur" in augs:
    aug_list.append(iaa.MedianBlur(k=(3,11)))
if "BilateralBlur" in augs:
    aug_list.append(iaa.BilateralBlur(
    d=(3, 10), sigma_color=(10, 250), sigma_space=(10, 250)))
if "MotionBlur" in augs:
    aug_list.append(iaa.MotionBlur(k=10,angle=[-45, 45]))
if "MeanShiftBlur" in augs:
    aug_list.append(iaa.MeanShiftBlur())
if "RandAugment" in augs: # 아무거나 적용하는 듯?
    aug_list.append(iaa.RandAugment(n=2,m=9))
    aug_list.append(iaa.RandAugment(m=30))
    aug_list.append(iaa.RandAugment(m=(0,9)))
if "WithColorspace" in augs:
    aug_list.append(iaa.WithColorspace(
    to_colorspace="HSV",
    from_colorspace="RGB",
    children=iaa.WithChannels(
        0,
        iaa.Add((0, 50))
    )
))
if "WithBrightnessChannels" in augs:
    aug_list.append(iaa.WithBrightnessChannels(iaa.Add((-50, 50))))
if "ChangeColorTemperature" in augs:
    aug_list.append(iaa.ChangeColorTemperature((1100, 10000)))
if "KMeansColorQuantization" in augs:
    aug_list.append(iaa.KMeansColorQuantization(n_colors=(4, 16)))
if "UniformColorQuantization" in augs:
    aug_list.append(iaa.UniformColorQuantization(n_colors=(4, 16)))
if "HorizontalFlip" in augs:
    aug_list.append(iaa.HorizontalFlip())
if "VerticalFlip" in augs:
    aug_list.append(iaa.VerticalFlip())
if "Fliplr" in augs:
    aug_list.append(iaa.Fliplr(0.5))
if "Fog" in augs:
    aug_list.append(iaa.imgcorruplike.Fog(severity=2))
if "Frost" in augs:
    aug_list.append(iaa.imgcorruplike.Frost(severity=2))
if "Snow" in augs:
    aug_list.append(iaa.imgcorruplike.Snow(severity=2))
if "Spatter" in augs:
    aug_list.append(iaa.imgcorruplike.Spatter(severity=2))
if "Pixelate" in augs:
    aug_list.append(iaa.imgcorruptlike.Pixelate(severity=2))
if "FastSnowyLandscape" in augs:
    aug_list.append(iaa.FastSnowyLandscape(
    lightness_threshold=[128, 200],
    lightness_multiplier=(1.5, 3.5)
))
if "Clouds" in augs:
    aug_list.append(iaa.Clouds())
if "Snowflakes" in augs:
    aug_list.append(flake_size=(0.2, 0.7), speed=(0.007, 0.03))
if "Rain" in augs:
    aug_list.append(iaa.Rain(speed=(0.1, 0.3)))
if "GammaContrast" in augs:
    aug_list.append(iaa.GammaContrast((0.5, 2.0)))
if "SigmoidContrast" in augs:
    aug_list.append(iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6)))
if "LogContrast" in augs:
    aug_list.append(iaa.LogContrast(gain=(0.6,1.4)))
if "LinearContrast" in augs:
    aug_list.append(iaa.LinearContrast((0.4,1.6)))
if "CKAGE" in augs:
    aug_list.append(iaa.AllChannelsCLAHE(clip_limit=(1, 10)))
if "AllChannelsHistogramEqualization" in augs:
    aug_list.append(iaa.Alpha((0.0, 1.0), iaa.AllChannelsHistogramEqualization()))
if "Convolve" in augs:
    matrix = np.array([[0, -1, 0],
                   [-1, 4, -1],
                   [0, -1, 0]])
    aug_list.append(iaa.Convolve(matrix=matrix))
if "Sharpen" in augs:
    aug_list.append(iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 2.0)))
if "Emboss" in augs:
    aug_list.append(iaa.Emboss(alpha=(0.0,1.0), strength=(0.5,1.5)))
if "Scale" in augs:
    aug_list.append(iaa.Affine(scale=(0.5, 1.5)))
if "Rotate" in augs:
    aug_list.append(iaa.Affine(rotate=(-45,45)))
if "Shear" in augs:
    aug_list.append(iaa.Affine(shear=(-16,16)))
if "PiecewiseAffine" in augs:
    aug_list.append( iaa.PiecewiseAffine(scale=(0.01, 0.05)))
if "PerspectiveTransform" in augs:
    aug_list.append(iaa.PerspectiveTransform(scale=(0.01, 0.15)))
if "ElasticTransformation" in augs:
    aug_list.append(iaa.ElasticTransformation(alpha=(0, 5.0), sigma=0.25))
if "CropAndPad" in augs:
    aug_list.append(iaa.CropAndPad(
        percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
    ))

In [11]:
augs

['Add', 'Dropout', 'MotionBlur']

# Augmentation 객체 생성

In [12]:
seq = iaa.Sequential( ## 선택된 augmentator 중에서 0~전체 개수의 augmentator에서 랜덤한 순서대로 선택하여 진행
    iaa.SomeOf((0, None),
              aug_list,
              random_order=True)
)
seq2 = iaa.Sequential( ## 하나만 랜덤하게 골라서 진행
    iaa.OneOf(aug_list)
)

# VOC 형식의 XML 데이터에서 바운딩 박스와 라벨 데이터 추출하는 함수

In [13]:
def getXMLDATA(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot() # 루트 파싱
    file_info ={}
    objects_list = []
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = root.find("size").find("width").text
    file_info['height'] = root.find("size").find("height").text
    #object 리스트 추가 
    for o in root.iter('object'):
        bndbox = {}
        bndbox['label'] = o.find("name").text # 라벨링 추가 
        sizes = []
        sizes.append(o.find("bndbox").find("xmin").text)
        sizes.append(o.find("bndbox").find("ymin").text)
        sizes.append(o.find("bndbox").find("xmax").text)
        sizes.append(o.find("bndbox").find("ymax").text)
        bndbox["bndbox"] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info

# yolov Darknet 의 txt 데이터에서 바운딩 박스와 라벨 데이터 추출

In [23]:
def getImgSize(img_path):
    image = Image.open(img_path)
    width, height = image.size
    size =[]
    size.append(width)
    size.append(height)
    return size

def getYoloDATA(file_path, img_size):
    with open(file_path, mode='r', encoding='utf-8') as f:
        datas=f.readlines()
    file_info = {}
    x = file_path.split('\\')
    x.reverse()
    #파일 명 추가
    file_info['filename'] = x[0]
    #사진 크기 추가
    file_info['width'] = img_size[0]
    file_info['height'] = img_size[1]
    #object 리스트 추가
    objects_list = []
    for i in datas:
        i = i.replace("\n","")
        bndbox = {}
        temp = i.split(' ')
        bndbox['label'] = temp[0]
        sizes = []
        xmax = (float(temp[1])*img_size[0]*2 + float(temp[3])*img_size[0])/2.0
        xmin = (float(temp[1])*img_size[0]*2 - float(temp[3])*img_size[0])/2.0
        ymax = (float(temp[2])*img_size[1]*2 + float(temp[4])*img_size[1])/2.0
        ymin = (float(temp[2])*img_size[1]*2 - float(temp[4])*img_size[1])/2.0
        sizes.append(xmin)
        sizes.append(ymin)
        sizes.append(xmax)
        sizes.append(ymax)
        bndbox['bndbox'] = sizes
        objects_list.append(bndbox)
    file_info['objects'] = objects_list
    return file_info
        
#         dw = 1./416
#         dh = 1./416
#         x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
#         y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
#         w = (int(box[1][0]) - int(box[0][0]))*dw
#         h = (int(box[1][1]) - int(box[0][1]))*dh     
    

# XML file Augmentation 진행 ->save yolo

In [15]:
if input_format == 'xml':
    for k in range(iter_num):
        for i, path in enumerate(xml_path_list):
            data = getXMLDATA(path)
            #확장자 버리고 이름만 따기
            name = data['filename'].split('.')
            name = name[0]
            #이미지 읽어오기
            image = imageio.imread(img_path_list[i])
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(416,416))

            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = 416/int(data["width"])
            y_ratio = 415/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(labelmap[obj['label']])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(int(obj['bndbox'][0])*x_ratio),
                                  y1=round(int(obj['bndbox'][1])*y_ratio),
                                  x2=round(int(obj['bndbox'][2])*x_ratio),
                                  y2=round(int(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)
            print(bbs_aug[0])

        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')


                for i,box in enumerate(bbs_aug):
                    dw = 1./416
                    dh = 1./416
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw
                    h = (int(box[1][1]) - int(box[0][1]))*dh
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_img+"\\"+name+'_aug' + str(k)+ 'txt', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
    for i in labellist:
        f.write(i+'\n')

# Yolo augmentation ->yolo save

In [26]:
if input_format == 'txt':
    for k in range(iter_num):
        for i, path in enumerate(txt_path_list):
            data = getYoloDATA(path,getImgSize(img_path_list[i]))
            #확장자 버리고 이름만 따기
            name = data['filename'].split('.')
            name = name[0]
            #이미지 읽어오기
            image = imageio.imread(img_path_list[i])
            #이미지 416 으로 리사이즈
            image = ia.imresize_single_image(image,(416,416))


            #resize 이후 라벨링 비율을 바꾸기 위함
            x_ratio = 416/int(data["width"])
            y_ratio = 416/int(data["height"])

            # 라벨 데이터와 바운딩 박스를 분리
            objects = data["objects"]

            text_list = []
            bbs_list = []

            for obj in objects:
                text_list.append(str(obj['label'])) #라벨들을 미리 숫자로 바꾸고 text 앞부분에 저장        
                bbs = BoundingBox(x1=round(int(obj['bndbox'][0])*x_ratio),
                                  y1=round(int(obj['bndbox'][1])*y_ratio),
                                  x2=round(int(obj['bndbox'][2])*x_ratio),
                                  y2=round(int(obj['bndbox'][3])*y_ratio))
                bbs_list.append(bbs)
        #         ia.imshow(bbs.draw_on_image(image, size=2)) # 이미지 출력해서 원본 이미지 라벨링 보고 싶다면 ㄱ
            #바운딩 박스 객체 생성
            bounding_box = BoundingBoxesOnImage(bbs_list, shape=image.shape)
            # augmentation 진행
            img_aug, bbs_aug = seq(image=image, bounding_boxes=bounding_box)


        #     ia.imshow(bbs_aug.draw_on_image(img_aug, size=2)) # 변환된 이미지 보고 싶다면 ㄱ

            # numpy 형식의 데이터인 이미지를 저장
            try: # 폴더 경로가 없는 경우 생성
                if not(os.path.isdir(save_img)):
                    os.makedirs(os.path.join(save_img))
                if not(os.path.isdir(save_label)):
                    os.makedirs(os.path.join(save_label))
            except OSError as e:
                if e.errno != errno.EXIST:
                    print("Failed to create directory!!!!!")
                    raise


            if save_format == "yolo":
                im = Image.fromarray(img_aug)
                im.save(save_img+"\\"+name+'_aug' + str(k)+ '.jpg')

                for i,box in enumerate(bbs_aug):
                    dw = 1./416
                    dh = 1./416
                    x = ((int(box[0][0])+int(box[1][0]))/2.0)*dw
                    y = ((int(box[0][1])+int(box[1][1]))/2.0)*dh
                    w = (int(box[1][0]) - int(box[0][0]))*dw
                    h = (int(box[1][1]) - int(box[0][1]))*dh
                    text_list[i] += ' ' + str(x) +' '+ str(y)  +' ' + str(w) +' ' +str(h)

                for i in text_list:
                    with open(save_img+"\\"+name+'_aug' + str(k)+ '.jpg', mode='wt', encoding='utf-8') as f:
                        f.write(i+'\n')

with open(save_img+'\\'+"classes.labels", mode='wt', encoding='utf-8') as f:
    for i in labellist:
        f.write(i+'\n')

BoundingBox(x1=149.0000, y1=103.0000, x2=288.0000, y2=385.0000, label=None)
BoundingBox(x1=139.0000, y1=91.0000, x2=286.0000, y2=380.0000, label=None)
BoundingBox(x1=145.0000, y1=100.0000, x2=290.0000, y2=379.0000, label=None)
BoundingBox(x1=147.0000, y1=101.0000, x2=288.0000, y2=378.0000, label=None)
BoundingBox(x1=149.0000, y1=103.0000, x2=288.0000, y2=385.0000, label=None)
BoundingBox(x1=151.0000, y1=104.0000, x2=285.0000, y2=384.0000, label=None)
BoundingBox(x1=148.0000, y1=103.0000, x2=287.0000, y2=385.0000, label=None)
BoundingBox(x1=149.0000, y1=102.0000, x2=285.0000, y2=377.0000, label=None)
BoundingBox(x1=149.0000, y1=102.0000, x2=285.0000, y2=377.0000, label=None)
BoundingBox(x1=278.0000, y1=153.0000, x2=352.0000, y2=257.0000, label=None)
BoundingBox(x1=280.0000, y1=150.0000, x2=350.0000, y2=252.0000, label=None)
BoundingBox(x1=279.0000, y1=148.0000, x2=351.0000, y2=251.0000, label=None)
BoundingBox(x1=277.0000, y1=146.0000, x2=351.0000, y2=250.0000, label=None)
BoundingBox(x

BoundingBox(x1=258.0000, y1=66.0000, x2=385.0000, y2=264.0000, label=None)
BoundingBox(x1=253.0000, y1=56.0000, x2=381.0000, y2=252.0000, label=None)
BoundingBox(x1=249.0000, y1=63.0000, x2=378.0000, y2=261.0000, label=None)
BoundingBox(x1=256.0000, y1=68.0000, x2=390.0000, y2=270.0000, label=None)
BoundingBox(x1=258.0000, y1=66.0000, x2=385.0000, y2=264.0000, label=None)
BoundingBox(x1=249.0000, y1=63.0000, x2=378.0000, y2=261.0000, label=None)
BoundingBox(x1=168.0000, y1=142.0000, x2=210.0000, y2=220.0000, label=None)
BoundingBox(x1=174.0000, y1=141.0000, x2=217.0000, y2=220.0000, label=None)
BoundingBox(x1=168.0000, y1=141.0000, x2=211.0000, y2=220.0000, label=None)
BoundingBox(x1=173.0000, y1=141.0000, x2=217.0000, y2=221.0000, label=None)
BoundingBox(x1=171.0000, y1=146.0000, x2=214.0000, y2=224.0000, label=None)
BoundingBox(x1=168.0000, y1=142.0000, x2=210.0000, y2=220.0000, label=None)
BoundingBox(x1=174.0000, y1=141.0000, x2=217.0000, y2=220.0000, label=None)
BoundingBox(x1=168

BoundingBox(x1=181.0000, y1=130.0000, x2=248.0000, y2=227.0000, label=None)
BoundingBox(x1=182.0000, y1=130.0000, x2=247.0000, y2=226.0000, label=None)
BoundingBox(x1=181.0000, y1=130.0000, x2=248.0000, y2=227.0000, label=None)
BoundingBox(x1=178.0000, y1=128.0000, x2=246.0000, y2=225.0000, label=None)
BoundingBox(x1=181.0000, y1=129.0000, x2=246.0000, y2=224.0000, label=None)
BoundingBox(x1=182.0000, y1=130.0000, x2=247.0000, y2=226.0000, label=None)
BoundingBox(x1=246.0000, y1=161.0000, x2=307.0000, y2=268.0000, label=None)
BoundingBox(x1=245.0000, y1=166.0000, x2=305.0000, y2=271.0000, label=None)
BoundingBox(x1=242.0000, y1=171.0000, x2=304.0000, y2=277.0000, label=None)
BoundingBox(x1=242.0000, y1=172.0000, x2=302.0000, y2=277.0000, label=None)
BoundingBox(x1=245.0000, y1=166.0000, x2=305.0000, y2=271.0000, label=None)
BoundingBox(x1=243.0000, y1=163.0000, x2=307.0000, y2=270.0000, label=None)
BoundingBox(x1=242.0000, y1=172.0000, x2=302.0000, y2=277.0000, label=None)
BoundingBox(

BoundingBox(x1=232.0000, y1=61.0000, x2=299.0000, y2=167.0000, label=None)
BoundingBox(x1=238.0000, y1=64.0000, x2=301.0000, y2=168.0000, label=None)
BoundingBox(x1=237.0000, y1=64.0000, x2=300.0000, y2=168.0000, label=None)
BoundingBox(x1=236.0000, y1=64.0000, x2=299.0000, y2=169.0000, label=None)
BoundingBox(x1=238.0000, y1=64.0000, x2=301.0000, y2=168.0000, label=None)
BoundingBox(x1=239.0000, y1=64.0000, x2=304.0000, y2=169.0000, label=None)
BoundingBox(x1=233.0000, y1=61.0000, x2=300.0000, y2=167.0000, label=None)
BoundingBox(x1=237.0000, y1=64.0000, x2=300.0000, y2=168.0000, label=None)
BoundingBox(x1=236.0000, y1=64.0000, x2=299.0000, y2=169.0000, label=None)
BoundingBox(x1=237.0000, y1=64.0000, x2=300.0000, y2=168.0000, label=None)
BoundingBox(x1=183.0000, y1=159.0000, x2=226.0000, y2=238.0000, label=None)
BoundingBox(x1=187.0000, y1=166.0000, x2=231.0000, y2=243.0000, label=None)
BoundingBox(x1=188.0000, y1=166.0000, x2=230.0000, y2=243.0000, label=None)
BoundingBox(x1=184.000

BoundingBox(x1=129.0000, y1=151.0000, x2=210.0000, y2=279.0000, label=None)
BoundingBox(x1=193.0000, y1=165.0000, x2=238.0000, y2=243.0000, label=None)
BoundingBox(x1=197.0000, y1=157.0000, x2=238.0000, y2=234.0000, label=None)
BoundingBox(x1=195.0000, y1=165.0000, x2=237.0000, y2=242.0000, label=None)
BoundingBox(x1=197.0000, y1=157.0000, x2=238.0000, y2=234.0000, label=None)
BoundingBox(x1=198.0000, y1=158.0000, x2=238.0000, y2=234.0000, label=None)
BoundingBox(x1=198.0000, y1=158.0000, x2=238.0000, y2=234.0000, label=None)
BoundingBox(x1=194.0000, y1=164.0000, x2=237.0000, y2=242.0000, label=None)
BoundingBox(x1=194.0000, y1=164.0000, x2=237.0000, y2=242.0000, label=None)
BoundingBox(x1=201.0000, y1=161.0000, x2=242.0000, y2=238.0000, label=None)
BoundingBox(x1=145.0000, y1=59.0000, x2=201.0000, y2=151.0000, label=None)
BoundingBox(x1=139.0000, y1=53.0000, x2=197.0000, y2=148.0000, label=None)
BoundingBox(x1=136.0000, y1=53.0000, x2=196.0000, y2=149.0000, label=None)
BoundingBox(x1=

...

BoundingBox(x1=178.0000, y1=141.0000, x2=236.0000, y2=235.0000, label=None)
BoundingBox(x1=125.0000, y1=136.0000, x2=195.0000, y2=275.0000, label=None)
BoundingBox(x1=122.0000, y1=133.0000, x2=197.0000, y2=274.0000, label=None)
BoundingBox(x1=122.0000, y1=133.0000, x2=197.0000, y2=274.0000, label=None)
BoundingBox(x1=121.0000, y1=136.0000, x2=189.0000, y2=275.0000, label=None)
BoundingBox(x1=128.0000, y1=137.0000, x2=200.0000, y2=279.0000, label=None)
BoundingBox(x1=122.0000, y1=133.0000, x2=197.0000, y2=274.0000, label=None)
BoundingBox(x1=159.0000, y1=97.0000, x2=265.0000, y2=295.0000, label=None)
BoundingBox(x1=162.0000, y1=98.0000, x2=282.0000, y2=305.0000, label=None)
BoundingBox(x1=166.0000, y1=100.0000, x2=279.0000, y2=303.0000, label=None)
BoundingBox(x1=166.0000, y1=100.0000, x2=276.0000, y2=302.0000, label=None)
BoundingBox(x1=157.0000, y1=96.0000, x2=266.0000, y2=296.0000, label=None)
BoundingBox(x1=164.0000, y1=99.0000, x2=281.0000, y2=304.0000, label=None)
BoundingBox(x1=1

BoundingBox(x1=186.0000, y1=81.0000, x2=310.0000, y2=289.0000, label=None)
BoundingBox(x1=225.0000, y1=147.0000, x2=327.0000, y2=308.0000, label=None)
BoundingBox(x1=221.0000, y1=147.0000, x2=320.0000, y2=304.0000, label=None)
BoundingBox(x1=223.0000, y1=149.0000, x2=320.0000, y2=305.0000, label=None)
BoundingBox(x1=220.0000, y1=146.0000, x2=321.0000, y2=303.0000, label=None)
BoundingBox(x1=222.0000, y1=147.0000, x2=322.0000, y2=305.0000, label=None)
BoundingBox(x1=225.0000, y1=147.0000, x2=327.0000, y2=308.0000, label=None)
BoundingBox(x1=222.0000, y1=149.0000, x2=318.0000, y2=304.0000, label=None)
BoundingBox(x1=224.0000, y1=145.0000, x2=329.0000, y2=308.0000, label=None)
BoundingBox(x1=223.0000, y1=149.0000, x2=320.0000, y2=305.0000, label=None)
BoundingBox(x1=223.0000, y1=149.0000, x2=320.0000, y2=305.0000, label=None)
BoundingBox(x1=224.0000, y1=145.0000, x2=329.0000, y2=308.0000, label=None)
BoundingBox(x1=222.0000, y1=149.0000, x2=318.0000, y2=304.0000, label=None)
BoundingBox(x

BoundingBox(x1=150.0000, y1=82.0000, x2=269.0000, y2=265.0000, label=None)
BoundingBox(x1=149.0000, y1=87.0000, x2=265.0000, y2=271.0000, label=None)
BoundingBox(x1=148.0000, y1=82.0000, x2=267.0000, y2=265.0000, label=None)
BoundingBox(x1=217.0000, y1=129.0000, x2=324.0000, y2=300.0000, label=None)
BoundingBox(x1=225.0000, y1=133.0000, x2=321.0000, y2=298.0000, label=None)
BoundingBox(x1=229.0000, y1=135.0000, x2=330.0000, y2=306.0000, label=None)
BoundingBox(x1=220.0000, y1=128.0000, x2=321.0000, y2=297.0000, label=None)
BoundingBox(x1=230.0000, y1=135.0000, x2=328.0000, y2=304.0000, label=None)
BoundingBox(x1=225.0000, y1=133.0000, x2=321.0000, y2=298.0000, label=None)
BoundingBox(x1=229.0000, y1=133.0000, x2=330.0000, y2=304.0000, label=None)
BoundingBox(x1=219.0000, y1=127.0000, x2=323.0000, y2=297.0000, label=None)
BoundingBox(x1=225.0000, y1=133.0000, x2=321.0000, y2=298.0000, label=None)
BoundingBox(x1=229.0000, y1=135.0000, x2=330.0000, y2=306.0000, label=None)
BoundingBox(x1=

BoundingBox(x1=174.0000, y1=146.0000, x2=261.0000, y2=291.0000, label=None)
BoundingBox(x1=180.0000, y1=145.0000, x2=265.0000, y2=285.0000, label=None)
BoundingBox(x1=176.0000, y1=147.0000, x2=260.0000, y2=291.0000, label=None)
BoundingBox(x1=180.0000, y1=143.0000, x2=265.0000, y2=284.0000, label=None)
BoundingBox(x1=171.0000, y1=141.0000, x2=262.0000, y2=290.0000, label=None)
BoundingBox(x1=179.0000, y1=138.0000, x2=240.0000, y2=231.0000, label=None)
BoundingBox(x1=181.0000, y1=138.0000, x2=242.0000, y2=231.0000, label=None)
BoundingBox(x1=180.0000, y1=139.0000, x2=240.0000, y2=231.0000, label=None)
BoundingBox(x1=179.0000, y1=147.0000, x2=236.0000, y2=236.0000, label=None)
BoundingBox(x1=178.0000, y1=146.0000, x2=237.0000, y2=236.0000, label=None)
BoundingBox(x1=177.0000, y1=146.0000, x2=236.0000, y2=236.0000, label=None)
BoundingBox(x1=179.0000, y1=147.0000, x2=236.0000, y2=236.0000, label=None)
BoundingBox(x1=184.0000, y1=149.0000, x2=240.0000, y2=240.0000, label=None)
BoundingBox(

PermissionError: [Errno 13] Permission denied: 'aug\\img\\TrafficLight_red320_jpg_aug.jpg'